In [2]:
import numpy as np
from tensorflow.keras.models import load_model
import pickle

# Load the trained model
model = load_model('models/computer_cnn_model.keras')

# Load the scaler
with open('models/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# Define a random input sample
# Example: Randomly generated values for a feature vector (adjust these values to your dataset range)
random_sample = np.array([[63, 1, 3, 145, 233, 1, 0, 150, 0, 2.3, 0, 0, 1]])  

# Scale the input using the same scaler used during training
random_sample_scaled = scaler.transform(random_sample)

# Reshape for the CNN model (3D input: samples, features, 1)
random_sample_scaled = random_sample_scaled.reshape(random_sample_scaled.shape[0], random_sample_scaled.shape[1], 1)

# Predict with the model
prediction = model.predict(random_sample_scaled)

# Interpret the prediction
predicted_class = (prediction.flatten() > 0.5).astype(int)

print("Prediction Probabilities:", prediction.flatten())
print("Predicted Class:", predicted_class[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Prediction Probabilities: [0.9969031]
Predicted Class: 1


In [13]:
# Positional Encoding Layer
@tf.keras.utils.register_keras_serializable()
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_len, embed_dim):
        super(PositionalEncoding, self).__init__()
        self.max_len = max_len
        self.embed_dim = embed_dim
        self.pos_encoding = self.positional_encoding(max_len, embed_dim)

    def positional_encoding(self, max_len, embed_dim):
        pos = np.arange(max_len)[:, np.newaxis]
        i = np.arange(embed_dim)[np.newaxis, :]
        angle_rads = pos / np.power(10000, (2 * (i // 2)) / np.float32(embed_dim))
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        pos_encoding = angle_rads[np.newaxis, ...]
        return tf.cast(pos_encoding, dtype=tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

    def get_config(self):
            config = super(PositionalEncoding, self).get_config()
            config.update({
                "max_len": self.max_len,
                "embed_dim": self.embed_dim,
            })
            return config

In [3]:
# Load the saved model
loaded_model = tf.keras.models.load_model("models/kaggle_transformer_model.keras", compile=False)

# Prepare the input data
data = np.array([63, 1, 3, 145, 233, 1, 0, 150, 0, 2.3, 0, 0, 1])
data = data.reshape(1, -1, 1)  # Reshape for model input [batch_size, sequence_length, feature_dim]

# Make the prediction
prediction_proba = loaded_model.predict(data).flatten()
prediction = (prediction_proba > 0.3).astype(int)

print("Prediction Probability:", prediction_proba[0])
print("Predicted Class:", prediction[0])


In [1]:
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load the saved scaler and model
with open('models/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('models/lgbm_model.pkl', 'rb') as f:
    lgbm_model = pickle.load(f)

# Sample test input (13 features)
test_input = np.array([[52, 1, 0, 125, 212, 0, 1, 168, 0, 1.0, 2, 2, 3]])

# Preprocess input
test_input_scaled = scaler.transform(test_input)

# Predict
probabilities = lgbm_model.predict_proba(test_input_scaled)[:, 1]
predictions = (probabilities > 0.5).astype(int)

# Output results
print(f"Probability of Heart Disease: {probabilities[0]:.4f}")
print(f"Prediction: {'Heart Disease' if predictions[0] == 1 else 'No Heart Disease'}")

UnpicklingError: invalid load key, '\x0b'.

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import os
from sklearn.preprocessing import StandardScaler
from joblib import load
from tensorflow.keras.models import load_model

# Define the expected feature names
expected_features = [
    'age', 'sex', 'chest_pain_type', 'resting_blood_pressure', 'cholesterol',
    'fasting_blood_sugar', 'resting_electrocardiogram', 'max_heart_rate_achieved',
    'exercise_induced_angina', 'st_depression', 'st_slope', 'num_major_vessels', 'thalassemia'
]

# Sample input (corrected to avoid duplicate keys)
sample_input = {
    'age': 52,
    'sex': 1,
    'chest_pain_type': 2,
    'resting_blood_pressure': 128,
    'cholesterol': 205,
    'fasting_blood_sugar': 0,
    'resting_electrocardiogram': 1,
    'max_heart_rate_achieved': 184,
    'exercise_induced_angina': 0,
    'st_depression': 0.0,
    'st_slope': 2,
    'num_major_vessels': 0,
    'thalassemia': 2
}

# --- LGBMClassifier Model ---
try:
    # Load the LGBMClassifier model
    lgbm_model = load('models/lgbm_model.pkl')

    # Convert sample input to DataFrame
    sample_df = pd.DataFrame([sample_input], columns=expected_features)

    # Predict
    prediction = lgbm_model.predict(sample_df)
    prediction_proba = lgbm_model.predict_proba(sample_df)

    # Output results
    print(f"Prediction (LGBM): {'Heart Disease' if prediction[0] == 1 else 'No Heart Disease'}")
    print(f"Prediction Probability (LGBM): {prediction_proba[0]}")
except Exception as e:
    print(f"Error loading or predicting with LGBM model: {e}")

Prediction (LGBM): Heart Disease
Prediction Probability (LGBM): [0.00215327 0.99784673]


Prediction (LGBM): Heart Disease
Prediction Probability (LGBM): [0.00215327 0.99784673]
